### Обычно курсоры автоматически закрываются при фиксации транзакции. Однако курсор, созданный как часть такого цикла,

### автоматически преобразуется в удерживаемый курсор при первом выполнении COMMIT или ROLLBACK.

### Это означает, что курсор полностью вычисляется при первом COMMIT или ROLLBACK, а не построчно.

### Курсор по-прежнему автоматически удаляется после цикла, поэтому это в основном невидимо для пользователя.

In [17]:
import psycopg2
import pandas as pd
import sqlalchemy

In [18]:
conn = psycopg2.connect(host="127.0.0.1", port="5432", database="Adventureworks", 
                            user="postgres", password="postgres")
cur = conn.cursor()

In [19]:
cur.execute('''
           create table if not exists  customer 
            (customer_id int,
             store_id int,
             first_name varchar,
             last_name varchar)
            ''')


cur.execute('''
           insert into   customer
            values(1,1,'Mike','Mike');
            ''')


cur.execute('''
           select* from customer
        
            ''')

results = cur.fetchall()
pd.DataFrame(results, columns=['customer_id', 'store_id', 'first_name', 'last_name'])

,customer_id,store_id,first_name,last_name
0,1,1,Mike,Mike


In [21]:
cur.execute('''
           create or replace function return_cursor() 
            returns refcursor as
            $$
            DECLARE
                res CONSTANT refcursor := '_result';
            begin
                open res for select * from customer;
                return res;

            end
            $$ language plpgsql;
            ''')

cur.execute('''
           begin;
            ''')

cur.execute('''
           begin;
            select * from return_cursor();
            fetch all from _result;
            ''')

results = cur.fetchall()
pd.DataFrame(results, columns=['customer_id', 'store_id', 'first_name', 'last_name'])

,customer_id,store_id,first_name,last_name
0,1,1,Mike,Mike


In [22]:
cur.execute('''
            -- получить последнего
            begin;
            select * from return_cursor();
            fetch last from _result;
            commit;
            --получить 4-го
            begin;
            select * from return_cursor();
            MOVE FORWARD 3 FROM _result;
            fetch next from _result;
            commit;
            --CLOSE закрывает связанный с курсором портал. 
            --Используется для того, чтобы освободить ресурсы раньше, чем закончится транзакция, 
            -- или чтобы освободить курсорную переменную для повторного открытия.
            ''')

In [16]:
conn.rollback()
conn.close()